In [28]:
%pip install pydantic-ai python-dotenv nest_asyncio
from IPython.display import clear_output ; clear_output()

from dotenv import load_dotenv ; load_dotenv(override=True)
import nest_asyncio ; nest_asyncio.apply()

import os
import asyncio
import random
from typing import List, Dict
from pprint import pprint, pformat

from pydantic import BaseModel, Field
from pydantic_ai import Agent

_ai_models = (
    [] + 
    (['openai:gpt-4o', 'openai:gpt-4o-mini'] if os.environ.get('OPENAI_API_KEY') else []) +
    (['gemini-1.5-pro', 'gemini-2.0-flash-exp'] if os.environ.get('GEMINI_API_KEY') else []) +
    (['claude-3-5-haiku-latest', 'claude-3-5-sonnet-latest'] if os.environ.get('ANTHROPIC_API_KEY') else [])
)
print(f"Available AI models: {pformat(_ai_models)}\n")

AI_MODEL = os.environ['AI_MODEL'] if os.environ.get('AI_MODEL') else random.choice(_ai_models)
print(f"Using AI model: {AI_MODEL}")

Available AI models: ['openai:gpt-4o',
 'openai:gpt-4o-mini',
 'gemini-1.5-pro',
 'gemini-2.0-flash-exp',
 'claude-3-5-haiku-latest',
 'claude-3-5-sonnet-latest']

Using AI model: openai:gpt-4o-mini


In [29]:
class Task(BaseModel):
    type: str = Field(..., description=(
        'The type of task. '
        'For example: "formal", "conversational", "hybrid", ...'))
    description: str = Field(..., description='Clear description for executing this task.')


class OrchestratorResponse(BaseModel):
    analysis: str = Field(..., description=(
        'Explain your understanding of the task and which variations would be valuable. '
        'Focus on how each approach serves different aspects of the task.'
    ))
    tasks: List[Task] = Field(..., description="List of tasks")


async def orchestrate(task: str) -> Dict:
    """Process task by breaking it down and running subtasks in parallel."""

    orchestrator_response = await Agent(
        AI_MODEL,
        system_prompt='Analyze this task and break it down into 2-3 distinct approaches.',
        result_type=OrchestratorResponse,
    ).run(task)
    
    # Parse orchestrator response
    analysis = orchestrator_response.data.analysis
    tasks = orchestrator_response.data.tasks
    
    print("\n=== ORCHESTRATOR OUTPUT ===")
    print(f"\nANALYSIS:\n{analysis}")
    print(f"\nTASKS:")
    for task in tasks:
        pprint(task.model_dump())
    
    # Step 2: Process all the tasks in parallel and collect results
    worker_agent = Agent(AI_MODEL, system_prompt='Generate content based on the task specification.')
    worker_responses = await asyncio.gather(*[
        worker_agent.run(pformat({'original_task': task} | task_info.model_dump()))
        for task_info in tasks
    ])
    worker_results = [
        {
            'type': task.type,
            'description': task.description,
            'result': response.data,
        }
        for task, response in zip(tasks, worker_responses)
    ]

    for worker_result in worker_results:
        print(f"\n=== WORKER RESULT ({worker_result['type']}) ===\n{worker_result['result']}\n")
    
    return {
        "analysis": analysis,
        "worker_results": worker_results,
    }

In [30]:
results = await orchestrate('Write a product description for a new eco-friendly water bottle.')


=== ORCHESTRATOR OUTPUT ===

ANALYSIS:
This task involves creating an appealing product description for a new eco-friendly water bottle. The approaches should focus on various selling points such as sustainability, functionality, and design aesthetics. Each approach serves to attract different customer segments that may prioritize distinct product features. For instance, environmentally conscious consumers may resonate more with sustainability aspects, while active lifestyle users may be drawn to functionality and portability.

TASKS:
{'description': 'Highlight the eco-friendly materials used, emphasize the '
                "bottle's recyclability, and discuss its impact on reducing "
                'plastic waste.',
 'type': 'sustainability-focused'}
{'description': "Describe the bottle's features such as leak-proof design, "
                'insulation properties, and capacity to cater to active users.',
 'type': 'functionality-focused'}
{'description': 'Focus on the design elemen